In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import os
import seaborn as sns

from matplotlib.colors import ListedColormap
from pathlib import Path
from digiforest_analysis.utils import plotting

### Parameters

In [ ]:
ROBOT_VELOCITY_THR = 0.01
SAFETY_VELOCITY_THR = 0.1
FILTERING_WINDOW = "5s"
BASE_INVERTED = True

In [ ]:
# Missions
missions = {
    "M1": {
        "path": "/media/matias/T71/datasets/2023-09-11-thesis-legged-forester-autonomy/2023-05-03-13-07-17-mission-m1"
    },
    "M2": {
        "path": "/media/matias/T71/datasets/2023-09-11-thesis-legged-forester-autonomy/2023-05-04-14-36-05-mission-m2"
    },
    "M3": {
        "path": "/media/matias/T71/datasets/2023-09-11-thesis-legged-forester-autonomy/2023-05-04-14-50-00-mission-m3"
    },
    "M4": {
        "path": "/media/matias/T71/datasets/2023-09-11-thesis-legged-forester-autonomy/2023-05-04-19-14-10-mission-m4"
    },
    "M5": {
        "path": "/media/matias/T71/datasets/2023-09-11-thesis-legged-forester-autonomy/2023-05-05-08-10-22-mission-m5"
    },
}

output_path = "/media/matias/T71/datasets/2023-09-11-thesis-legged-forester-autonomy/"

## Matplotlib config

In [ ]:
cm = 1 / 2.54
plot_width = 8.89 * cm
plot_height = 8 * cm

plt.rcParams["font.size"] = 8

n_colors = 10

### Load pose files

In [ ]:
from scipy.spatial.transform import Rotation as R


def read_poses_file(filename, base_inverted=False):
    df = pd.read_csv(filename)
    df = df.drop_duplicates()

    # Generate timestamp from sec and nsec
    ts = 1e9 * df["sec"] + df["nsec"]  # In nanoseconds
    df.index = pd.to_datetime(ts)
    df = df[~df.index.duplicated(keep="first")]

    # Parse data
    poses = {}
    for ts, x, y, z, qx, qy, qz, qw in zip(
        df.index, df["x"], df["y"], df["z"], df["qx"], df["qy"], df["qz"], df["qw"]
    ):
        poses[f"{ts:.10f}"] = np.eye(4)
        poses[f"{ts:.10f}"][0:3, 3] = np.array([x, y, z])
        poses[f"{ts:.10f}"][0:3, 0:3] = R.from_quat([qx, qy, qz, qw]).as_matrix()

        # TODO: fix base inversion

    return df, poses

### Load twist files

In [ ]:
def read_twist_file(filename, base_inverted=False):
    df = pd.read_csv(filename)
    df = df.drop_duplicates()

    # Generate timestamp from sec and nsec
    ts = 1e9 * df["sec"] + df["nsec"]  # In nanoseconds
    df.index = pd.to_datetime(ts)
    df = df[~df.index.duplicated(keep="first")]

    # Correct twist due to base inversion
    if BASE_INVERTED:
        df["vx"] *= -1
        df["vy"] *= -1

    # Speeds
    df["lin_speed"] = (df["vx"] ** 2 + df["vy"] ** 2).pow(1.0 / 2)
    df["ang_speed"] = df["wz"].abs()

    return df

### Load other operator's signals

In [ ]:
def read_param_change_file(filename, base_inverted=False):
    df = pd.read_csv(filename)
    df = df.drop_duplicates()

    # Generate timestamp from sec and nsec
    ts = 1e9 * df["sec"] + df["nsec"]  # In nanoseconds
    df.index = pd.to_datetime(ts)
    df = df[~df.index.duplicated(keep="first")]

    return df

### Read signals

In [ ]:
for k, m in missions.items():
    mission_path = m["path"]

    missions[k]["df_state_poses"], _ = read_poses_file(
        os.path.join(mission_path, "states/state_pose_data.csv"),
        base_inverted=BASE_INVERTED,
    )
    missions[k]["df_state_twist"] = read_twist_file(
        os.path.join(mission_path, "states/state_twist_data.csv"),
        base_inverted=BASE_INVERTED,
    )
    missions[k]["df_reference_twist"] = read_twist_file(
        os.path.join(mission_path, "states/reference_twist_data.csv"),
        base_inverted=BASE_INVERTED,
    )
    missions[k]["df_operator_twist"] = read_twist_file(
        os.path.join(mission_path, "states/operator_twist_data.csv"),
        base_inverted=BASE_INVERTED,
    )
    missions[k]["df_local_planner_param"] = read_param_change_file(
        os.path.join(mission_path, "states/local_planner_param_data.csv"),
    )
    missions[k]["df_rmp_param"] = read_param_change_file(
        os.path.join(mission_path, "states/rmp_param_data.csv"),
    )

### Align time series

In [ ]:
for k, m in missions.items():
    # Extend indices
    joined_indices = (
        missions[k]["df_state_twist"]
        .index.union(missions[k]["df_reference_twist"].index)
        .drop_duplicates()
    )
    joined_indices = joined_indices.union(
        missions[k]["df_operator_twist"].index
    ).drop_duplicates()
    joined_indices = joined_indices.union(
        missions[k]["df_local_planner_param"].index
    ).drop_duplicates()
    joined_indices = joined_indices.union(
        missions[k]["df_rmp_param"].index
    ).drop_duplicates()

    missions[k]["df_state_twist"] = missions[k]["df_state_twist"].reindex(
        index=joined_indices
    )
    missions[k]["df_state_twist"] = missions[k]["df_state_twist"].interpolate(
        method="index"
    )

    missions[k]["df_operator_twist"] = missions[k]["df_operator_twist"].reindex(
        index=joined_indices
    )
    missions[k]["df_operator_twist"] = missions[k]["df_operator_twist"].interpolate(
        method="index"
    )

    missions[k]["df_reference_twist"] = missions[k]["df_reference_twist"].reindex(
        index=joined_indices
    )
    missions[k]["df_reference_twist"] = missions[k]["df_reference_twist"].interpolate(
        method="index"
    )

    missions[k]["df_local_planner_param"] = missions[k][
        "df_local_planner_param"
    ].reindex(index=joined_indices)
    missions[k]["df_rmp_param"] = missions[k]["df_rmp_param"].reindex(
        index=joined_indices
    )

## Estimate metrics

### Operator interventions
This is required for autonomy metrics

In [ ]:
for k, m in missions.items():
    # Time in motion
    missions[k]["df_state_twist"]["in_motion"] = (
        missions[k]["df_state_twist"]["lin_speed"] > ROBOT_VELOCITY_THR
    )

    # Interventions from safety officer
    missions[k]["df_state_twist"]["safety_intervention"] = (
        missions[k]["df_operator_twist"]["lin_speed"] > SAFETY_VELOCITY_THR
    )
    missions[k]["df_state_twist"]["safety_intervention"] = missions[k][
        "df_state_twist"
    ]["safety_intervention"].interpolate(method="pad")

    # Interventions from forestry operator
    missions[k]["df_state_twist"]["operator_intervention"] = (
        ~missions[k]["df_local_planner_param"]["sec"].isna()
    ) + (~missions[k]["df_rmp_param"]["sec"].isna())

### Normalize time

In [ ]:
for k, m in missions.items():
    ref_ts = missions[k]["df_state_twist"].index[0]

    missions[k]["df_state_poses"].index = missions[k]["df_state_poses"].index - ref_ts
    missions[k]["df_state_twist"].index = missions[k]["df_state_twist"].index - ref_ts
    missions[k]["df_reference_twist"].index = (
        missions[k]["df_reference_twist"].index - ref_ts
    )
    missions[k]["df_operator_twist"].index = (
        missions[k]["df_operator_twist"].index - ref_ts
    )

## Plots

### Autonomy

In [ ]:
from scipy import signal

plot_width = 15 * cm
plot_height = 7 * cm

N = len(missions)

fig, ax = plt.subplots(
    1,
    1,
    figsize=(plot_width, plot_height),
    constrained_layout=False,
    dpi=300,
    sharex=True,
)

# Axes
ax.set_axisbelow(True)
ax.grid(zorder=-1)
ax.grid(which="major", color=plotting.gray_palette_str["20"], linewidth=0.7)
ax.grid(
    which="minor", color=plotting.gray_palette_str["10"], linestyle=":", linewidth=0.5
)
ax.minorticks_on()

yticks_val = []
yticks_label = []
legend_handles = []
legend_labels = []

for i, (k, v) in enumerate(missions.items()):
    t = missions[k]["df_state_twist"].index.total_seconds()
    offset = -i * 0.5

    yticks_val.append(offset)
    yticks_label.append(k)

    # Plot standing time
    line_standing = ax.fill_between(
        t,
        offset - 0.05,
        offset + 0.05,
        linewidth=0,
        color=plotting.gray_palette_str["40"],
        alpha=0.7,
        label="Standing",
        zorder=1,
    )

    # Plot walking time
    in_motion = missions[k]["df_state_twist"]["in_motion"]
    line_walking = ax.fill_between(
        t,
        in_motion * (offset - 0.07),
        in_motion * (offset + 0.07),
        linewidth=0,
        color=plotting.color_palette_str["blue"],
        alpha=1.0,
        label="Walking",
        zorder=1,
    )

    # Plot raw interventions
    line_raw_int = ax.fill_between(
        t,
        missions[k]["df_state_twist"]["safety_intervention"] * (offset - 0.02),
        missions[k]["df_state_twist"]["safety_intervention"] * (offset + 0.02),
        linewidth=0,
        color="k",
        alpha=1.0,
        label="Safety op. action",
        zorder=5,
    )

    # Plot filtered interventions
    # Filter signal
    filt = t * 0  # Just initial value
    filt = (
        missions[k]["df_state_twist"]["safety_intervention"]
        .rolling(FILTERING_WINDOW, center=True)
        .max()
        > 0.5
    )
    missions[k]["df_state_twist"]["safety_intervention_filt"] = filt
    line_int = ax.fill_between(
        t,
        filt * (offset - 0.1),
        filt * (offset + 0.1),
        linewidth=0,
        color=plotting.color_palette_str["orange"],
        alpha=1.0,
        label="Intervention",
        zorder=4,
    )

    # We find the peaks in the original interventions signal, that should correspond to the center
    peaks, properties = signal.find_peaks(filt, distance=1)
    missions[k]["safety_intervention_num"] = len(peaks)
    # ax.plot(t[peaks], missions[k]["df_state_twist"]["operator_intervention"][peaks] + offset, ".", linewidth=0.1)

    # We find the left and right edges of the peak
    results_half = signal.peak_widths(
        missions[k]["df_state_twist"]["safety_intervention_filt"], peaks, rel_height=0.5
    )
    pos_l = results_half[2].astype(int)
    pos_r = results_half[3].astype(int)
    # ax.plot(t[pos_l], missions[k]["df_state_twist"]["operator_intervention"][pos_l] + offset, ".", linewidth=0.1, color="r")
    # ax.plot(t[pos_r], missions[k]["df_state_twist"]["operator_intervention"][pos_r] + offset, ".", linewidth=0.1, color="b")

    # Intervention time
    missions[k]["interventions"] = []
    for l, r in zip(pos_l, pos_r):
        dt = t[r] - t[l]
        missions[k]["interventions"].append(dt)

    # Autonomy time
    # We need to do some hacking here
    if (len(pos_l) and len(pos_r)) or len(peaks) == 0:
        pos_auto_l = pos_r
        pos_auto_r = pos_l
        pos_auto_l = np.insert(pos_auto_l, 0, 0, axis=0)
        pos_auto_r = np.insert(pos_auto_r, len(pos_auto_r), len(t) - 1, axis=0)
        # ax.plot(t[pos_auto_l], t[pos_auto_l]*0 + offset, ".", linewidth=0.1, color="m")
        # ax.plot(t[pos_auto_r], t[pos_auto_r]*0 + offset, ".", linewidth=0.1, color="c")

    missions[k]["autonomy"] = []
    # print(f"{k}")
    for l, r in zip(pos_auto_l, pos_auto_r):
        dt = t[r] - t[l]
        # print(f"t[{r}] - t[{l}] = {t[r]} - {t[l]} = {dt}")
        missions[k]["autonomy"].append(dt)
    
    if i == 0:
        legend_handles = [line_standing, line_walking, line_raw_int, line_int]
        legend_labels = [h.get_label() for h in legend_handles]

lgnd = ax.legend(legend_handles, legend_labels, edgecolor=(1, 1, 1, 0), framealpha=0.9, loc=(0.0, 1.01), ncol=4)
for line in lgnd.get_lines():
        line.set_sizes([40.0])

# Set grid
loc = plt.MultipleLocator(10.0)  # this locator puts ticks at regular intervals
# ax.xaxis.set_major_locator(loc)
ax.yaxis.set_major_locator(loc)
ax.margins(x=0.05)
ax.set_yticks(yticks_val, yticks_label, rotation=0)
ax.set_xlabel("Time [s]")

fig.set_tight_layout(True)
fig.savefig(os.path.join(output_path, "mission_autonomy_summary.pdf"))

In [ ]:
print("Number of interventions")
for k, v in missions.items():
    print(f"{k}: {v['safety_intervention_num']}")

import scipy.integrate as integrate

print("")
print("Autonomy percentage")
for k, v in missions.items():
    moving_seconds = integrate.simpson(
        missions[k]["df_state_twist"]["in_motion"],
        missions[k]["df_state_twist"]["in_motion"].index.total_seconds(),
    ).item()

    intervention_seconds = integrate.simpson(
        missions[k]["df_state_twist"]["safety_intervention_filt"],
        missions[k]["df_state_twist"]["safety_intervention_filt"].index.total_seconds(),
    ).item()

    mission_time = missions[k]["df_state_twist"].index.total_seconds()[-1]
    autonomy_perc = 100 * (1.0 - intervention_seconds / (moving_seconds))
    mission_autonomy_perc = 100 * (1.0 - intervention_seconds / (mission_time))

    print(f"{k}: {autonomy_perc:.2f}, {mission_autonomy_perc:.2f}")

### Interventions plot

In [ ]:
accumulated_interventions = []
list_interventions = []
color_interventions = []
label_interventions = []
for i, (k, m) in enumerate(missions.items()):
    accumulated_interventions.extend(m["interventions"])
    list_interventions.append(np.asarray(m["interventions"], dtype=int))
    color_interventions.append(plotting.color_palette[i])
    label_interventions.append(k)
accumulated_interventions = np.asarray(accumulated_interventions, dtype=int)

# Plotting settings
plot_width = 12 * cm
plot_height = 6 * cm

# Plot
fig, ax = plt.subplots(
    1,
    1,
    figsize=(plot_width, plot_height),
    constrained_layout=False,
    dpi=300,
    sharex=True,
)
# Axes
ax.set_axisbelow(True)
# ax.set_aspect("equal")
ax.grid(which="major", color=plotting.gray_palette_str["20"], linewidth=0.7)
ax.grid(
    which="minor", color=plotting.gray_palette_str["10"], linestyle=":", linewidth=0.5
)
ax.minorticks_on()
# Plot
ax.hist(
    list_interventions,
    bins=20,
    cumulative=False,
    density=False,
    stacked=True,
    color=color_interventions,
    label=label_interventions,
    rwidth=0.9,
)
# ax.set_title("Distribution of intervention time")
ax.set_xlabel("Intervention duration [s]")
ax.set_ylabel("Frequency")

lgnd = ax.legend(edgecolor=(1, 1, 1, 0), framealpha=0.9, loc=(0.09, 1.01), ncol=5)
for handle in lgnd.legend_handles:
    try:
        handle.set_sizes([40.0])
    except Exception as e:
        print(e)

fig.set_tight_layout(True)
fig.savefig(os.path.join(output_path, "intervention_distribution.pdf"))

### Autonomy plot

In [ ]:
accumulated_autonomy = []
list_autonomy = []
color_autonomy = []
label_autonomy = []
for i, (k, m) in enumerate(missions.items()):
    accumulated_autonomy.extend(m["autonomy"])
    list_autonomy.append(np.asarray(m["autonomy"], dtype=int))
    color_autonomy.append(plotting.color_palette[i])
    label_autonomy.append(k)
accumulated_autonomy = np.asarray(accumulated_autonomy, dtype=int)

# Plotting settings
plot_width = 12 * cm
plot_height = 6 * cm

# Plot
fig, ax = plt.subplots(
    1,
    1,
    figsize=(plot_width, plot_height),
    constrained_layout=False,
    dpi=300,
    sharex=True,
)
# Axes
ax.set_axisbelow(True)
# ax.set_aspect("equal")
ax.grid(which="major", color=plotting.gray_palette_str["20"], linewidth=0.7)
ax.grid(
    which="minor", color=plotting.gray_palette_str["10"], linestyle=":", linewidth=0.5
)
ax.minorticks_on()
# Plot
ax.hist(
    list_autonomy,
    bins=20,
    cumulative=False,
    density=False,
    stacked=True,
    log=False,
    color=color_autonomy,
    label=label_autonomy,
    rwidth=0.9,
)
# ax.set_title("Distribution of intervention time")
ax.set_xlabel("Time between interventions [s]")
ax.set_ylabel("Frequency")
ax.set_ylim([0, 16])

lgnd = ax.legend(edgecolor=(1, 1, 1, 0), framealpha=0.9, loc=(0.09, 1.01), ncol=5)
for handle in lgnd.legend_handles:
    try:
        handle.set_sizes([40.0])
    except Exception as e:
        print(e)

fig.set_tight_layout(True)
fig.savefig(os.path.join(output_path, "time_between_interventions_distribution.pdf"))

In [ ]:
# Plot
fig, ax = plt.subplots(
    1,
    1,
    figsize=(plot_width, plot_height),
    constrained_layout=False,
    dpi=300,
    sharex=True,
)
# Axes
ax.set_axisbelow(True)
# ax.set_aspect("equal")
ax.grid(which="major", color=plotting.gray_palette_str["20"], linewidth=0.7)
ax.grid(
    which="minor", color=plotting.gray_palette_str["10"], linestyle=":", linewidth=0.5
)
ax.minorticks_on()
# Plot

_, bins = np.histogram(np.log10(accumulated_autonomy + 1), bins=20)

ax.hist(
    list_autonomy,
    bins=10**bins,
    cumulative=False,
    density=False,
    stacked=True,
    log=False,
    color=color_autonomy,
    label=label_autonomy,
    rwidth=0.9,
)
# ax.set_title("Distribution of intervention time")
ax.set_xlabel("Time between interventions [s]")
ax.set_ylabel("Frequency")
ax.set_xscale("log")

lgnd = ax.legend(edgecolor=(1, 1, 1, 0), framealpha=0.9, loc=(0.09, 1.01), ncol=5)
for line in lgnd.get_lines():
    line.set_sizes([40.0])

fig.set_tight_layout(True)
fig.savefig(os.path.join(output_path, "time_between_interventions_distribution_log.pdf"))

### Robot velocity

In [ ]:
smoothing_window = "3000ms"
linewidth = 0.75

plot_width = 15 * cm
plot_height = 4 * cm

N = len(missions)


for i, (k, m) in enumerate(missions.items()):
    fig, ax = plt.subplots(
        1,
        1,
        figsize=(plot_width, plot_height),
        constrained_layout=False,
        dpi=300,
        sharex=True,
    )

    # Axes
    ax.set_axisbelow(True)
    # ax.set_aspect("equal")
    ax.grid(which="major", color=plotting.gray_palette_str["20"], linewidth=0.7)
    ax.grid(
        which="minor",
        color=plotting.gray_palette_str["10"],
        linestyle=":",
        linewidth=0.5,
    )
    ax.minorticks_on()

    ax.plot(
        missions[k]["df_state_twist"].index.total_seconds(),
        missions[k]["df_state_twist"]["lin_speed"]
        .rolling(smoothing_window, center=True)
        .mean(),
        label="Robot speed",
        linewidth=linewidth,
        color=plotting.color_palette_str["blue"],
    )
    ax.plot(
        missions[k]["df_reference_twist"].index.total_seconds(),
        missions[k]["df_reference_twist"]["lin_speed"]
        .rolling(smoothing_window, center=True)
        .mean(),
        label="Local planner reference",
        linewidth=linewidth,
        color=plotting.gray_palette_str["60"],
    )

    # if missions[k]["df_state_twist"]["operator_intervention"].any():
    #     ax[i].fill_between(
    #         missions[k]["df_state_twist"]["operator_intervention"].index.total_seconds(),
    #         missions[k]["df_state_twist"]["operator_intervention"].index.total_seconds() * 0,
    #         missions[k]["df_state_twist"]["operator_intervention"],
    #         label="Operator action",
    #         linewidth=1,
    #         linestyle="-",
    #         color=plotting.gray_palette_str["90"],
    #         alpha=1.0,
    #     )

    if missions[k]["df_state_twist"]["safety_intervention_filt"].any():
        ax.fill_between(
            missions[k]["df_state_twist"][
                "safety_intervention_filt"
            ].index.total_seconds(),
            missions[k]["df_state_twist"][
                "safety_intervention_filt"
            ].index.total_seconds()
            * 0,
            missions[k]["df_state_twist"]["safety_intervention_filt"],
            label="Intervention",
            linewidth=0,
            color=plotting.color_palette_str["orange"],
            alpha=0.7,
        )

    # Legends
    lgnd = ax.legend(edgecolor=(1, 1, 1, 0), framealpha=0.9, loc=(0, 1.03), ncol=3)
    for handle in lgnd.legend_handles:
        try:
            handle.set_sizes([40.0])
        except Exception as e:
            print(e)

    for handle in lgnd.get_lines():
        handle.set_linewidth(1)

    # ax.set_title('Computation Time')
    ax.margins(x=0, y=0)
    ax.set_ylabel("Speed [m/s]")
    ax.set_xlabel("Time [s]")

    # Export
    fig.set_tight_layout(True)
    fig.savefig(os.path.join(output_path, f"mission_velocity_{k.lower()}.pdf"))